In [142]:
#Import Pandas to parse the dataframe
import pandas as pd

#Used to download the sampledata module
#import bokeh as bo
#bo.sampledata.download()

#Import the necessary Bokeh tools
from bokeh.plotting import figure, show
from bokeh.layouts import column
from bokeh.models import WheelPanTool, WheelZoomTool, Range1d, Scatter, ColumnDataSource
import bokeh.sampledata.stocks as stocks

#Get the data
df = pd.DataFrame(stocks.MSFT)[60:120]
df["date"] = pd.to_datetime(df["date"])

dates = df.date.to_list()

#Data for the candlestick graph
candlestick_data = df
inc = df.close > df.open
dec = df.open > df.close
w = 16*60*60*1000 # milliseconds

#Data for the chevrons
chevron_dec = dec.to_list()
def chevron_filter(property, value): 
    output = []
    for x in range(len(chevron_dec)):
        if chevron_dec[x] != value: #XOR operator
            output.append(df[property].to_list()[x])
    return output

#Data for the line graph
line_data = df

#Define the tools on the graph (Has been replaced)
#TOOLS = "xwheel_zoom"

#Create a function to make the figures
def makeplot(is_line, is_candlestick, range):
    #Create the figure
    p = figure(x_axis_type="datetime", width=1000, height=150,
           title="Graph", background_fill_color="#efefef", tools="", toolbar_location=None)
    
    p.sizing_mode="inherit"
    p.xaxis.major_label_orientation = 0.8 # radians
    zoom = WheelZoomTool(dimensions="width")
    p.add_tools(zoom)
    p.toolbar.active_scroll = zoom


    #Configure its properties
    if range != 0: 
        p.x_range=range
    else:
        pan = WheelPanTool(dimension="width")
        p.add_tools(pan)
        p.x_range = Range1d(dates[0], dates[-1], bounds=(dates[0], dates[-1]))
        p.toolbar_location = "above"
    
    p.title = f'MSFT {"candlestick" if is_candlestick else ""}{" + " if is_candlestick and is_line else ""}{"line" if is_line else ""}'

    #Add the candlestick graph
    if is_candlestick:
        p.segment(candlestick_data.date, candlestick_data.high, candlestick_data.date, candlestick_data.low, color="black")

        p.vbar(candlestick_data.date[dec], w, candlestick_data.open[dec], candlestick_data.close[dec], fill_color="#eb3c40")
        p.vbar(candlestick_data.date[inc], w, candlestick_data.open[inc], candlestick_data.close[inc], fill_color="white",
        line_color="#49a3a3", line_width=2)

        white_scatter = Scatter(x="x", y="y", size=10, marker="triangle", fill_color="white")
        white_source = ColumnDataSource({"x": chevron_filter("date", True), "y": chevron_filter("high", True)})
        p.add_glyph(white_source, white_scatter)

        red_scatter = Scatter(x="x", y="y", size=10, marker="inverted_triangle", fill_color="#eb3c40")
        red_source = ColumnDataSource({"x": chevron_filter("date", False), "y": chevron_filter("high", False)})
        p.add_glyph(red_source, red_scatter)

    #Add the line graph
    if is_line:
        p.line(line_data.date, line_data.high, line_width=2)
    return p

#Make the figures
candlestick_line = makeplot(True, True, 0)
candlestick = makeplot(False, True, candlestick_line.x_range)
line = makeplot(True, False, candlestick_line.x_range)

#Display the figures
display_column=column(candlestick_line, candlestick, line)
display_column.sizing_mode="stretch_both"
show(display_column)